In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re

In [3]:
corps_list = pd.read_csv('list.csv', dtype=object)
corps_codes = corps_list['종목코드']

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
 
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)

In [5]:
def xp(xpath):
    return driver.find_element(by=By.XPATH, value = xpath)
 

def get_report(code):
    url = f"https://dart.fss.or.kr/navi/searchNavi.do?naviCrpNm={code}&naviCode=A002"
    driver.get(url)
    time.sleep(1)

    try:
        driver.switch_to.frame("naviIfrm_1")

        return driver.page_source

    except:
        return 0

In [6]:
def pre_processing(page):
    soup = BeautifulSoup(page, 'html.parser')
    current = soup.find('a',{"name":"toc1"}).parent
    end = soup.find('a',{"name":"toc3"}).parent

    cleaned_text = ""
    cleaned_table = []

    while current != end:
        next_element = current.next_sibling

        if current.name == 'div':
            pass
        elif current.name == 'table':
            tb = ""
            if current.get('class') == ['nb']:
                tb += current.get_text(strip=True) + '\n'
                # 다음 요소가 테이블인지 확인
                next_element = current.find_next_sibling()
                while next_element and next_element.name == 'table':
                    if next_element.get('class') == ['nb']:
                        tb += next_element.get_text(strip=True) + '\n'
                    else:
                        tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(next_element)) + '\n'
                    next_element = next_element.find_next_sibling()
            else:
                tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(current)) + '\n'
                next_element = current.find_next_sibling()
                while next_element and next_element.name == 'table':
                    if next_element.get('class') == ['nb']:
                        tb += next_element.get_text(strip=True) + '\n'
                    else:
                        tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(next_element)) + '\n'
                    next_element = next_element.find_next_sibling()
            
            # Add the cleaned table content to the list
            cleaned_table.append(tb)

        else:
            # Process non-table tags to only extract text content
            text = current.get_text(strip=True)
            if text:
                cleaned_text += text + '\n'

        # Move to the next element for further processing
        current = next_element
        
    return cleaned_text, cleaned_table

In [ ]:
import json

data = dict()
for idx, code in enumerate(corps_codes):
    page = get_report(code)
    if page == 0:
        pass
    else:
        cleaned_text, cleaned_table = pre_processing(page)
        data[code] = {"text": cleaned_text, "table": cleaned_table}
        print(f"{idx+1}/{len(corps_codes)} {code} done")
    
    if idx % 10 == 0:
        with open('data.json', 'w') as f:
            json.dumps(data, ensure_ascii=False)

In [ ]:
page = get_report('044990')
soup = BeautifulSoup(page, 'html.parser')

current = soup.find('a',{"name":"toc1"}).parent
end = soup.find('a',{"name":"toc3"}).parent

cleaned_text = ""
cleaned_table = []

while current != end:
    next_element = current.next_sibling

    if current.name == 'div':
        pass
    elif current.name == 'table':
        tb = ""
        if current.get('class') == ['nb']:
            tb += current.get_text(strip=True) + '\n'
            # 다음 요소가 테이블인지 확인
            next_element = current.find_next_sibling()
            while next_element and next_element.name == 'table':
                if next_element.get('class') == ['nb']:
                    tb += next_element.get_text(strip=True) + '\n'
                else:
                    tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(next_element)) + '\n'
                next_element = next_element.find_next_sibling()
        else:
            tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(current)) + '\n'
            next_element = current.find_next_sibling()
            while next_element and next_element.name == 'table':
                if next_element.get('class') == ['nb']:
                    tb += next_element.get_text(strip=True) + '\n'
                else:
                    tb += re.sub(r'<(\w+)(?:\s+[^>]*)?>', r'<\1>', str(next_element)) + '\n'
                next_element = next_element.find_next_sibling()
        
        # Add the cleaned table content to the list
        cleaned_table.append(tb)

    else:
        # Process non-table tags to only extract text content
        text = current.get_text(strip=True)
        if text:
            cleaned_text += text + '\n'

    # Move to the next element for further processing
    current = next_element

In [ ]:
cleaned_table